In [5]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
import sklearn.metrics as mt
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from imblearn.combine import SMOTEENN

le = LabelEncoder()
# Importing the dataset
dataset=pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/club-churn.csv')
#filling missing value with mean
dataset['points_in_wallet'].fillna(value=dataset['points_in_wallet'].mean(), inplace=True)
dataset['avg_frequency_login_days'] = pd.to_numeric(dataset['avg_frequency_login_days'], errors='coerce').fillna(0)
#transform data into numeric value
dataset['gender']=le.fit_transform(dataset['gender'])
dataset['region_category']=le.fit_transform(dataset['region_category'])
dataset['membership_category']=le.fit_transform(dataset['membership_category'])
dataset['joined_through_referral']=le.fit_transform(dataset['joined_through_referral'])
dataset['preferred_offer_types']=le.fit_transform(dataset['preferred_offer_types'])
dataset['medium_of_operation']=le.fit_transform(dataset['medium_of_operation'])
dataset['internet_option']=le.fit_transform(dataset['internet_option'])
dataset['used_special_discount']=le.fit_transform(dataset['used_special_discount'])
dataset['offer_application_preference']=le.fit_transform(dataset['offer_application_preference'])
dataset['past_complaint']=le.fit_transform(dataset['past_complaint'])
dataset['complaint_status']=le.fit_transform(dataset['complaint_status'])
dataset['feedback']=le.fit_transform(dataset['feedback'])


dataset=dataset.dropna()


X=dataset.drop(['churn_risk_score','security_no','joining_date','referral_id','last_visit_time'],axis=1)
y=dataset["churn_risk_score"]

#imblearn SMOTE
#smote_enn = SMOTE()
smote_enn = SMOTEENN()
X, y = smote_enn.fit_resample(X, y)

#smote_enn = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
#X, y = smote_enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)


#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)


#Training XGBoost
#classifier = XGBClassifier(eta=0.3, max_depth = 4, gamma=0, min_child_weight=1)
classifier = XGBClassifier(colsample_bytree=0.6, max_depth = 5, gamma=1.5, min_child_weight=5)

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============XGBoost Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

===============XGBoost Performance====================
F1 score: 0.984
STD F1 Score: 0.002
Recall: 0.988
Specitifity: 0.980
AUC ROC: 0.999
G-Mean: 0.984


In [6]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators =100, criterion = 'gini', random_state = 0)

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============Random Forest Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

===============Random Forest Performance====================
F1 score: 0.982
STD F1 Score: 0.001
Recall: 0.982
Specitifity: 0.982
AUC ROC: 0.999
G-Mean: 0.982


In [7]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier(n_estimators=100,learning_rate=0.1,algorithm='SAMME.R')

score=cross_val_score(classifier, X, y, scoring='f1', cv=cv, n_jobs=-1)
print('===============ADABoost Performance====================')

print('F1 score: %.3f' % (mean(score)))
print('STD F1 Score: %.3f' % (std(score)))

rc=cross_val_score(classifier, X, y, scoring='recall', cv=cv, n_jobs=-1)
print('Recall: %.3f' % (mean(rc)))

from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score

# Define the specificity scorer
scorer = make_scorer(specificity_score)

# Calculate the cross-validated specificity score
sp = cross_val_score(classifier, X, y, cv=5, scoring=scorer)
print('Specitifity: %.3f' % (mean(sp)))

auc=cross_val_score(classifier, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('AUC ROC: %.3f' % (mean(auc)))

import math

gmean=rc*sp
gmean=mean(gmean)
sqrtg = math.sqrt(gmean)
print('G-Mean: %.3f' % (sqrtg))
print('======================================================')

===============ADABoost Performance====================
F1 score: 0.970
STD F1 Score: 0.004
Recall: 0.969
Specitifity: 0.972
AUC ROC: 0.997
G-Mean: 0.970
